# NEST Simulator in Google Colab

This notebook allows you to run NEST Simulator using **Google Colab's free computational resources**.

## What You Get:
- ✅ Free CPU/RAM from Google Colab
- ✅ No installation on your local machine
- ✅ Access from any device with a browser
- ✅ Easy sharing and collaboration

## Quick Start:
1. Run the installation cell below (takes ~2-3 minutes)
2. Run the example simulations
3. Modify and experiment!

---

## Step 1: Install NEST Simulator

Run this cell to install NEST using conda. This takes about 2-3 minutes.

In [ ]:
%%bash
# Install conda if not already available
if ! command -v conda &> /dev/null; then
    echo "Installing Miniconda..."
    wget -q https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O miniconda.sh
    bash miniconda.sh -b -p $HOME/miniconda
    rm miniconda.sh
    export PATH="$HOME/miniconda/bin:$PATH"
    conda init bash
fi

# Install NEST Simulator from conda-forge
echo "Installing NEST Simulator..."
conda install -y -c conda-forge nest-simulator -q

echo "\n✅ NEST installation complete!"

## Step 2: Verify Installation

Check that NEST is properly installed and import it.

In [ ]:
import nest
import matplotlib.pyplot as plt
import numpy as np

print(f"✅ NEST Simulator version: {nest.__version__}")
print(f"✅ Available neuron models: {len(nest.node_models)}")
print(f"✅ Available synapse models: {len(nest.synapse_models)}")
print("\n🎉 NEST is ready to use!")

## Example 1: Single Neuron Simulation

Simple example of simulating a single integrate-and-fire neuron with constant current injection.

In [ ]:
# Reset NEST kernel
nest.ResetKernel()

# Create a single integrate-and-fire neuron
neuron = nest.Create('iaf_psc_alpha')

# Create a constant current generator
dc_generator = nest.Create('dc_generator', params={'amplitude': 500.0})

# Create a voltmeter to record membrane potential
voltmeter = nest.Create('voltmeter')

# Create a spike recorder
spike_recorder = nest.Create('spike_recorder')

# Connect devices to neuron
nest.Connect(dc_generator, neuron)
nest.Connect(voltmeter, neuron)
nest.Connect(neuron, spike_recorder)

# Simulate for 100ms
nest.Simulate(100.0)

# Get recorded data
vm_times = voltmeter.events['times']
vm_values = voltmeter.events['V_m']
spike_times = spike_recorder.events['times']

# Plot results
fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(vm_times, vm_values, 'b-', linewidth=2)
ax.set_xlabel('Time (ms)', fontsize=12)
ax.set_ylabel('Membrane Potential (mV)', fontsize=12)
ax.set_title('Single Neuron with Constant Current Injection', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)

# Mark spike times
for spike_time in spike_times:
    ax.axvline(spike_time, color='r', linestyle='--', alpha=0.5, linewidth=1)

plt.tight_layout()
plt.show()

print(f"\n📊 Number of spikes: {len(spike_times)}")
print(f"📊 First spike at: {spike_times[0]:.2f} ms")
if len(spike_times) > 1:
    mean_isi = np.mean(np.diff(spike_times))
    firing_rate = 1000.0 / mean_isi
    print(f"📊 Mean firing rate: {firing_rate:.2f} Hz")

## Example 2: Small Network - Excitatory and Inhibitory Neurons

Simulate a small network with excitatory and inhibitory neurons.

In [ ]:
# Reset NEST kernel
nest.ResetKernel()

# Network parameters
n_excitatory = 80  # Number of excitatory neurons
n_inhibitory = 20  # Number of inhibitory neurons
simulation_time = 1000.0  # ms

# Create neurons
excitatory = nest.Create('iaf_psc_alpha', n_excitatory)
inhibitory = nest.Create('iaf_psc_alpha', n_inhibitory)

# Create Poisson noise generators for background activity
noise = nest.Create('poisson_generator', params={'rate': 8000.0})

# Create spike recorders
spike_recorder_exc = nest.Create('spike_recorder')
spike_recorder_inh = nest.Create('spike_recorder')

# Synapse parameters
syn_exc = {'weight': 50.0, 'delay': 1.0}
syn_inh = {'weight': -200.0, 'delay': 1.0}

# Connect noise to all neurons
nest.Connect(noise, excitatory + inhibitory, syn_spec={'weight': 30.0, 'delay': 1.0})

# Connect excitatory to all neurons (E→E and E→I)
nest.Connect(excitatory, excitatory + inhibitory, 
             conn_spec={'rule': 'pairwise_bernoulli', 'p': 0.1},
             syn_spec=syn_exc)

# Connect inhibitory to all neurons (I→E and I→I)
nest.Connect(inhibitory, excitatory + inhibitory,
             conn_spec={'rule': 'pairwise_bernoulli', 'p': 0.1},
             syn_spec=syn_inh)

# Connect spike recorders
nest.Connect(excitatory, spike_recorder_exc)
nest.Connect(inhibitory, spike_recorder_inh)

# Simulate
print("🔄 Simulating network...")
nest.Simulate(simulation_time)
print("✅ Simulation complete!")

# Get spike data
exc_events = spike_recorder_exc.events
inh_events = spike_recorder_inh.events

# Create raster plot
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

# Plot excitatory neurons
ax1.scatter(exc_events['times'], exc_events['senders'], 
            s=2, c='blue', alpha=0.6, label='Excitatory')
ax1.set_ylabel('Neuron ID', fontsize=12)
ax1.set_title('Excitatory Neurons', fontsize=12, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.legend()

# Plot inhibitory neurons
ax2.scatter(inh_events['times'], inh_events['senders'],
            s=2, c='red', alpha=0.6, label='Inhibitory')
ax2.set_xlabel('Time (ms)', fontsize=12)
ax2.set_ylabel('Neuron ID', fontsize=12)
ax2.set_title('Inhibitory Neurons', fontsize=12, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.legend()

plt.tight_layout()
plt.show()

# Calculate and display statistics
n_spikes_exc = len(exc_events['times'])
n_spikes_inh = len(inh_events['times'])
rate_exc = n_spikes_exc / (n_excitatory * simulation_time / 1000.0)
rate_inh = n_spikes_inh / (n_inhibitory * simulation_time / 1000.0)

print(f"\n📊 Network Statistics:")
print(f"  Excitatory neurons: {n_excitatory}")
print(f"  Inhibitory neurons: {n_inhibitory}")
print(f"  Total spikes (E): {n_spikes_exc}")
print(f"  Total spikes (I): {n_spikes_inh}")
print(f"  Mean firing rate (E): {rate_exc:.2f} Hz")
print(f"  Mean firing rate (I): {rate_inh:.2f} Hz")

## Example 3: Balanced Random Network (Brunel Model)

Implementation of a balanced random network as described in Brunel (2000).

In [ ]:
# Reset NEST kernel
nest.ResetKernel()
nest.set_verbosity('M_WARNING')

# Network parameters (scaled down for Colab)
N_neurons = 1000      # Total number of neurons
gamma = 0.25          # Ratio of inhibitory to excitatory neurons
N_E = int(N_neurons * (1 - gamma))  # Excitatory neurons
N_I = int(N_neurons * gamma)         # Inhibitory neurons

# Connectivity
C_E = int(0.1 * N_E)  # Number of excitatory synapses per neuron
C_I = int(0.1 * N_I)  # Number of inhibitory synapses per neuron

# Synapse parameters
J_E = 0.1    # mV (excitatory synaptic weight)
g = 5.0      # Relative inhibitory to excitatory synaptic weight
J_I = -g * J_E
delay = 1.5  # ms (synaptic delay)

# External input
nu_ext = 8.0  # kHz (external input rate)
p_rate = nu_ext * 1000.0  # Convert to Hz

# Simulation parameters
sim_time = 1000.0  # ms

print("🔧 Building network...")
print(f"  E neurons: {N_E}, I neurons: {N_I}")
print(f"  Connections per neuron: {C_E} (E), {C_I} (I)")

# Create neuron populations
neurons_E = nest.Create('iaf_psc_delta', N_E)
neurons_I = nest.Create('iaf_psc_delta', N_I)
all_neurons = neurons_E + neurons_I

# Create external input
poisson_generator = nest.Create('poisson_generator', params={'rate': p_rate})

# Create spike recorder (record from subset for visualization)
spike_recorder = nest.Create('spike_recorder')

# Connect Poisson input to all neurons
nest.Connect(poisson_generator, all_neurons, syn_spec={'weight': J_E, 'delay': delay})

# Connect E→E and E→I
print("🔗 Connecting excitatory neurons...")
nest.Connect(neurons_E, all_neurons,
             conn_spec={'rule': 'fixed_indegree', 'indegree': C_E},
             syn_spec={'weight': J_E, 'delay': delay})

# Connect I→E and I→I
print("🔗 Connecting inhibitory neurons...")
nest.Connect(neurons_I, all_neurons,
             conn_spec={'rule': 'fixed_indegree', 'indegree': C_I},
             syn_spec={'weight': J_I, 'delay': delay})

# Connect spike recorder to first 100 neurons
nest.Connect(all_neurons[:100], spike_recorder)

# Simulate
print("🔄 Simulating...")
nest.Simulate(sim_time)
print("✅ Simulation complete!")

# Get spike data
events = spike_recorder.events
spike_times = events['times']
spike_senders = events['senders']

# Plot raster plot
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8), 
                                gridspec_kw={'height_ratios': [3, 1]})

# Raster plot
ax1.scatter(spike_times, spike_senders, s=1, c='black', alpha=0.5)
ax1.set_ylabel('Neuron ID', fontsize=12)
ax1.set_title('Balanced Random Network - Raster Plot', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)

# Population firing rate (histogram)
bin_size = 10.0  # ms
bins = np.arange(0, sim_time + bin_size, bin_size)
hist, _ = np.histogram(spike_times, bins=bins)
rate = hist / (bin_size / 1000.0) / 100  # Convert to Hz
bin_centers = bins[:-1] + bin_size / 2

ax2.plot(bin_centers, rate, 'b-', linewidth=2)
ax2.set_xlabel('Time (ms)', fontsize=12)
ax2.set_ylabel('Population Rate (Hz)', fontsize=12)
ax2.set_title('Population Firing Rate', fontsize=12, fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Statistics
total_spikes = len(spike_times)
mean_rate = total_spikes / (100 * sim_time / 1000.0)

print(f"\n📊 Network Statistics:")
print(f"  Total neurons: {N_neurons} (E: {N_E}, I: {N_I})")
print(f"  Total spikes recorded: {total_spikes}")
print(f"  Mean firing rate: {mean_rate:.2f} Hz")
print(f"  E/I ratio: {1/gamma:.2f}")

## Additional Resources

### NEST Documentation
- [NEST Homepage](https://nest-simulator.org)
- [NEST Documentation](https://nest-simulator.readthedocs.io)
- [PyNEST Tutorial](https://nest-simulator.readthedocs.io/en/stable/tutorials/index.html)
- [Example Networks](https://nest-simulator.readthedocs.io/en/stable/examples/index.html)

### Available Models
You can list all available neuron and synapse models:

In [ ]:
import nest

print("🧠 Neuron Models:")
neuron_models = [m for m in nest.node_models if 'generator' not in m and 'recorder' not in m]
for i, model in enumerate(sorted(neuron_models[:20]), 1):  # Show first 20
    print(f"  {i:2d}. {model}")
print(f"\n  ... and {len(neuron_models) - 20} more models\n")

print("🔗 Synapse Models:")
for i, model in enumerate(sorted(nest.synapse_models[:15]), 1):  # Show first 15
    print(f"  {i:2d}. {model}")
print(f"\n  ... and {len(nest.synapse_models) - 15} more models")

## Tips for Using NEST in Colab

### Resource Management
- **Free tier limits**: Colab has RAM and runtime limits (~12GB RAM, ~12 hours)
- **Scale your simulations**: Start with smaller networks, then scale up
- **Save your work**: Download important results before session expires

### Performance Tips
1. Use `nest.set_verbosity('M_WARNING')` to reduce output
2. Record from subsets of neurons, not the entire network
3. Use `nest.SetKernelStatus({'local_num_threads': 2})` for parallel processing

### Saving Results
You can save plots and data to Google Drive:

In [ ]:
# Mount Google Drive (optional)
from google.colab import drive
# drive.mount('/content/drive')

# Save figure example
# plt.savefig('/content/drive/MyDrive/nest_simulation.png', dpi=300, bbox_inches='tight')

# Save data example
# np.save('/content/drive/MyDrive/spike_data.npy', spike_times)

print("💾 Uncomment the code above to save files to Google Drive")

## Next Steps

Try modifying the examples above:
- Change network sizes and connectivity parameters
- Explore different neuron models (e.g., `aeif_cond_alpha`, `hh_psc_alpha`)
- Implement synaptic plasticity (STDP, STP)
- Create spatially structured networks
- Analyze network dynamics and statistics

Happy simulating! 🧠⚡